# Анализ данных и процессов
## Лабораторная работа №1. Пьяница на утесе.
### Исходные данные.
Пьяница стоит между двумя пропастями, с одной стороны река, с другой копья. В начальный момент времени пьяница стоит на левой ноге. Его поведение задается графом марковского процесса (смотри приложенный рисунок). 

![](media/drunker.jpg)

### Задание 1. Необходимо определить среднее время жизни пьяницы и вероятность упасть в реку.

In [49]:
from typing import Counter

import numpy as np
import pandas as pd

i, j, k = 2, 3, 1

p = np.array([
    [1, 0, 0, 0, 0],
    [0, 1, 0, 0, 0],
    [i / (i + j), 0, 0, j / (i + j), 0],
    [0, 0, i / (i + j + k), j / (i + j + k), k / (i + j + k)],
    [0, k / (j + k), 0, j / (j + k), 0]
])

LABELS = ["Река", "Копья", "Левая", "Обе", "Правая"]
pd.DataFrame(p, index=LABELS, columns=LABELS)

,Река,Копья,Левая,Обе,Правая
Река,1.0,0.00,0.000000,0.00,0.000000
Копья,0.0,1.00,0.000000,0.00,0.000000
Левая,0.4,0.00,0.000000,0.60,0.000000
Обе,0.0,0.00,0.333333,0.50,0.166667
Правая,0.0,0.25,0.000000,0.75,0.000000


#### Получение фундаментальной матрицы

In [50]:
q = p[2:5, 2:5]
pd.DataFrame(q, index=LABELS[2:5], columns=LABELS[2:5])

,Левая,Обе,Правая
Левая,0.000000,0.60,0.000000
Обе,0.333333,0.50,0.166667
Правая,0.000000,0.75,0.000000


In [51]:
n = np.linalg.inv(np.identity(len(q)) - q)
pd.DataFrame(n, index=LABELS[2:5], columns=LABELS[2:5])

,Левая,Обе,Правая
Левая,2.142857,3.428571,0.571429
Обе,1.904762,5.714286,0.952381
Правая,1.428571,4.285714,1.714286


#### Среднее время жизни при старте с левой ноги

In [52]:
print(sum(n[0]))

6.142857142857141


#### Вероятность упасть в реку

In [53]:
r = p[2:5, 0:2]
pd.DataFrame(r, index=LABELS[2:5], columns=LABELS[0:2])

,Река,Копья
Левая,0.4,0.00
Обе,0.0,0.00
Правая,0.0,0.25


In [54]:
b = np.dot(n, r)
pd.DataFrame(b, index=LABELS[2:5], columns=LABELS[0:2])

,Река,Копья
Левая,0.857143,0.142857
Обе,0.761905,0.238095
Правая,0.571429,0.428571



### Задание 2. Написать программу, которая имитирует поведение пьяницы и выводит среднее количество переходов до падения с утеса и долю падений в реку.

#### Класс симулятора

In [55]:
from collections import Counter


class Simulator:
    def __init__(self, initial_state: int, terminate_states: list[int]):
        self._initial_state = initial_state
        self._terminate_states = terminate_states

    def __call__(self, move_matrix: np.ndarray, iters: int) -> tuple[int, Counter]:
        steps_count = 0
        end_state_counts = Counter(self._terminate_states)

        for _ in range(iters):
            current_state, steps = self._simulate_steps(move_matrix)
            steps_count += steps
            end_state_counts[current_state] += 1

        return steps_count, end_state_counts

    def _simulate_steps(self, move_matrix: np.ndarray):
        current_state, steps_count = self._initial_state, 0

        while current_state not in self._terminate_states:
            current_state = self._change_state(move_matrix, current_state)
            steps_count += 1

        return current_state, steps_count

    @staticmethod
    def _change_state(move_matrix: np.ndarray, current_state: int) -> int:
        seed, cumulative_probability = np.random.random(), 0

        for index, probability in enumerate(move_matrix[current_state]):
            cumulative_probability += probability
            if seed < cumulative_probability:
                return index
            

#### Имитация поведения пьяницы

In [56]:
simulator = Simulator(initial_state=2, terminate_states=[0, 1])
N = 100000

step_count, end_state_counts = simulator(p, N)

average_steps = step_count / N
print(f"Среднее количество шагов до падения: {average_steps:.2f}")
print("Процент падений:")

for state_index, count in end_state_counts.items():
    percentage = (count / N) * 100
    print(f"{LABELS[state_index]}: {count} ({percentage:.2f}%)")

Среднее количество шагов до падения: 6.17
Процент падений:
Река: 85641 (85.64%)
Копья: 14361 (14.36%)


### Задание 3. Сравнить теоретическую вероятность падению в реку с долей падения в реку критерием сравнения долей.

In [57]:
THRESHOLD = 1.96

p_exp = b[0][0]
p_fact = end_state_counts[0] / N

k = np.abs((p_fact - p_exp) / ((p_fact * (1 - p_fact) / N) ** 0.5))

if k < THRESHOLD:
    print("Гипотеза принимается.")
else:
    print("Гипотеза отвергается")

Гипотеза принимается.


## Вывод
В ходе выполнения лабораторной работы была решена задача "Пьяница на утесе" с помощью цепей Маркова. Сначала были построены все необходимые матрицы, затем рассчитано среднее количество переходов до падения с утеса и доля падений в реку, проведено сравнение теоретической вероятности падения с данными полученными в ходе симуляции.